In [1]:
import os

# windows記得先載visual studio c++
import cv2
import json
# import openslide
import matplotlib.pyplot as plt
import large_image
import numpy as np
import skimage.io
from skimage import io
# import large_image_source_openslide
from PIL import Image
# from openslide.deepzoom import DeepZoomGenerator
import matplotlib.image as mpimg
from tqdm import tqdm_notebook
import math
import pandas as pd
import MTM  # multi template matching
from MTM import matchTemplates, drawBoxesOnRGB
from shapely.geometry import Polygon
from scipy import stats
import glob
from pandas import json_normalize
from tqdm import tqdm

Image.MAX_IMAGE_PIXELS = 10000000000

In [2]:
ellipse = pd.read_excel("./template_excel/ellipse_detail_BFC_0515.xlsx")

In [3]:
convex = pd.read_excel("./template_excel/convex_detail_PTC_20230630.xlsx")

In [5]:
ellipse.loc[ellipse['img_name'].str.contains("CYST"),"type"] = "CYST"
ellipse.loc[ellipse['img_name'].str.contains("HT"),"type"] = "HT"
# ellipse.loc[ellipse['img_name'].str.contains("PTC"),"type"] = "PTC"
ellipse.loc[ellipse['type'].isna(),"type"] = "Normal"
ellipse = ellipse.drop("Unnamed: 0",axis=1).reset_index(drop=True)

In [6]:
ellipse.describe()

,threshold,angle,area_cv2,slide_mean,contours_mean,contours_number
count,43114.000000,43114.000000,43114.000000,43114.000000,43114.000000,43114.000000
mean,134.273670,88.552770,470.489887,184.100292,123.817889,38.160969
std,8.434276,52.031468,106.550747,18.967301,5.169930,7.119249
min,115.000000,0.000000,38.000000,51.000000,93.351323,20.000000
25%,130.000000,43.480600,397.500000,173.000000,120.614525,33.000000
50%,130.000000,87.485199,456.500000,187.000000,124.468943,38.000000
75%,140.000000,133.782665,526.500000,198.000000,127.648081,43.000000
max,150.000000,179.990936,1117.000000,225.000000,158.201923,75.000000


In [11]:
ellipse[ellipse["type"]=="Normal"].describe()

,threshold,angle,area_cv2,slide_mean,contours_mean,contours_number
count,15881.000000,15881.000000,15881.000000,15881.000000,15881.000000,15881.000000
mean,134.900195,87.340980,459.795731,180.371702,122.781797,36.760280
std,8.669400,52.351247,93.992930,18.877507,5.129078,5.704777
min,125.000000,0.017474,38.000000,51.000000,93.351323,20.000000
25%,125.000000,41.187588,397.500000,169.000000,119.610577,33.000000
50%,140.000000,85.220627,452.000000,181.000000,123.679012,37.000000
75%,140.000000,132.801102,511.500000,194.000000,126.931667,41.000000
max,145.000000,179.977631,1117.000000,225.000000,130.000000,50.000000


In [61]:
def ellipse_template(image_path,folder_all, folder_cell, folder_excel):
    detail = pd.DataFrame()
    for second_dir in os.listdir(image_path):  # slide folder  
        for image_name in os.listdir(image_path + second_dir):
            if "BFC" in image_name:
                try:
                    o = cv2.imread(image_path + second_dir + "/" + image_name)
                    gray = cv2.cvtColor(o , cv2.COLOR_BGR2GRAY)
                    ret , binary = cv2.threshold(gray , 145 , 255 , cv2.THRESH_BINARY) # ret = threshold
                    contours , hierarchy = cv2.findContours(binary , cv2.RETR_LIST , cv2.CHAIN_APPROX_SIMPLE)
                    for i in range(len(contours)):
                        if len(contours[i]) >= 5 and len(contours[i]) <= 75 :
                            ellipse = cv2.fitEllipse(contours[i])
                            if 15 < ellipse[1][0] < 40 and 15 < ellipse[1][1] < 40: 
                                print(image_name.split(".png")[0] + "_" + str(i),"的 ellipse = " , ellipse[1])
                                result_ellipse = cv2.ellipse(o , ellipse , (255,255,255))

                                # get the parameter of the small rectangle
                                center, size, angle = ellipse[0], ellipse[1], ellipse[2]
                                center, size = tuple(map(int, center)), tuple(map(int, size))

                                # get row and col num in img
                                height, width = result_ellipse.shape[0], result_ellipse.shape[1]

                                # calculate the rotation matrix
                                M = cv2.getRotationMatrix2D(center, angle, 1)
                                # rotate the original image
                                img_rot = cv2.warpAffine(result_ellipse, M, (width, height))

                                # now rotated rectangle becomes vertical and we crop it
                                img_crop = cv2.getRectSubPix(img_rot, size, center)

                                # if img_crop.mean() <= 127:

                                ellipse_data = {
                                    "mass_name" : image_name.split(".png")[0],
                                    "img_name" : image_name.split(".png")[0] + "_" + str(i),
                                    "threshold" : ret,
                                    "center" : ellipse[0], 
                                    "axle_short" : ellipse[1][0],
                                    "axle_long" : ellipse[1][1],
                                    "ratio" : ellipse[1][1] / ellipse[1][0],
                                    "angle" : ellipse[2], 
                                    "area_cv2" : cv2.contourArea(contours[i]), 
                                    "perimeter" : cv2.arcLength(contours[i],True),
                                    "contours_pixel" : img_crop.mean(), 
                                    "contours_number" : len(contours[i]), 
                                    # "type" : "Normal", 
                                    "calcularity" : (4*3.14*cv2.contourArea(contours[i]))/ (cv2.arcLength(contours[i],True)*cv2.arcLength(contours[i],True))
                                }
                                detail = pd.concat([detail,pd.DataFrame([ellipse_data])],ignore_index=True)

                                cv2.imwrite(folder_all + image_name, result_ellipse)

                                cv2.imwrite(folder_cell + image_name.split(".png")[0] + "_" + str(i) + ".png", img_crop)

#                         elif len(contours[i]) > 41: 
#                             ellipse = cv2.fitEllipse(contours[i])
#                             if 18 < ellipse[1][0] < 40 and 18 < ellipse[1][1] < 40: 
#                                 print(image_name.split(".png")[0] + "_" + str(i),"的 ellipse = " , ellipse[1])
#                                 result_ellipse = cv2.ellipse(o , ellipse , (255,255,255))

#                                 # get the parameter of the small rectangle
#                                 center, size, angle = ellipse[0], ellipse[1], ellipse[2]
#                                 center, size = tuple(map(int, center)), tuple(map(int, size))

#                                 # get row and col num in img
#                                 height, width = result_ellipse.shape[0], result_ellipse.shape[1]

#                                 # calculate the rotation matrix
#                                 M = cv2.getRotationMatrix2D(center, angle, 1)
#                                 # rotate the original image
#                                 img_rot = cv2.warpAffine(result_ellipse, M, (width, height))

#                                 # now rotated rectangle becomes vertical and we crop it
#                                 img_crop = cv2.getRectSubPix(img_rot, size, center)

#                                 if (img_crop.mean() > 130 and cv2.contourArea(contours[i] < 500) or (img_crop.mean() < 130 and cv2.contourArea(contours[i] > 500):
#                                     ellipse_data = {
#                                         "img_name" : image_name.split(".png")[0] + "_" + str(i),
#                                         "threshold" : ret,
#                                         "center" : ellipse[0], 
#                                         "axle_short" : ellipse[1][0],
#                                         "axle_long" : ellipse[1][1],
#                                         "ratio" : ellipse[1][1] / ellipse[1][0],
#                                         "angle" : ellipse[2], 
#                                         "area_cv2" : cv2.contourArea(contours[i]), 
#                                         "perimeter" : cv2.arcLength(contours[i],True),
#                                         "contours_pixel" : img_crop.mean(), 
#                                         "contours_number" : len(contours[i]), 
#                                         "type" : "CYST", 
#                                         "calcularity" : (4*3.14*cv2.contourArea(contours[i]))/ (cv2.arcLength(contours[i],True)*cv2.arcLength(contours[i],True))
#                                     }
#                                     detail = pd.concat([detail,pd.DataFrame([ellipse_data])],ignore_index=True)

#                                     cv2.imwrite(folder_all + image_name, result_ellipse)

#                                     cv2.imwrite(folder_cell + image_name.split(".png")[0] + "_" + str(i) + ".png", img_crop)
#                                 else:
#                                     ellipse_data = {
#                                         "img_name" : image_name.split(".png")[0] + "_" + str(i),
#                                         "threshold" : ret,
#                                         "center" : ellipse[0], 
#                                         "axle_short" : ellipse[1][0],
#                                         "axle_long" : ellipse[1][1],
#                                         "ratio" : ellipse[1][1] / ellipse[1][0],
#                                         "angle" : ellipse[2], 
#                                         "area_cv2" : cv2.contourArea(contours[i]), 
#                                         "perimeter" : cv2.arcLength(contours[i],True),
#                                         "contours_pixel" : img_crop.mean(), 
#                                         "contours_number" : len(contours[i]), 
#                                         "type" : "HT", 
#                                         "calcularity" : (4*3.14*cv2.contourArea(contours[i]))/ (cv2.arcLength(contours[i],True)*cv2.arcLength(contours[i],True))
#                                     }
#                                     detail = pd.concat([detail,pd.DataFrame([ellipse_data])],ignore_index=True)

#                                     cv2.imwrite(folder_all + image_name, result_ellipse)

#                                     cv2.imwrite(folder_cell + image_name.split(".png")[0] + "_" + str(i) + ".png", img_crop)
                        else:
                            print(f'{image_name.split(".png")[0] + "_" + str(i)} : contours_number = {len(contours[i])}')
                except Exception as e :
                    print(f'{image_name} : error is {e}')
    detail.to_excel(folder_excel +"cut_BFC_0710_new.xlsx")

In [ ]:
ellipse_template("./MTM2023/","./cut_by_detail/BFC/all_new/","./cut_by_detail/BFC/singal_new/","./cut_by_detail/BFC/excel/")

BFC_20_015HT_44_0 : contours_number = 1
BFC_20_015HT_44_2 的 ellipse =  (29.356182098388672, 30.6442813873291)
BFC_20_015HT_44_5 : contours_number = 4
BFC_20_015HT_44_6 : contours_number = 4
BFC_20_015HT_44_7 : contours_number = 117
BFC_20_015HT_44_9 : contours_number = 1
BFC_20_015HT_44_10 : contours_number = 4
BFC_20_015HT_44_11 : contours_number = 1
BFC_20_015HT_44_15 : contours_number = 1
BFC_20_015HT_44_16 : contours_number = 4
BFC_20_015HT_44_17 的 ellipse =  (28.1910400390625, 32.252994537353516)
BFC_20_015HT_44_18 : contours_number = 4
BFC_20_015HT_44_20 的 ellipse =  (23.725866317749023, 27.023273468017578)
BFC_20_015HT_44_21 : contours_number = 235
BFC_20_015HT_44_23 : contours_number = 4
BFC_20_015HT_60_2 : contours_number = 3
BFC_20_015HT_60_5 : contours_number = 4
BFC_20_015HT_60_6 : contours_number = 171
BFC_20_015HT_60_8 的 ellipse =  (20.488977432250977, 35.40903854370117)
BFC_20_015HT_60_9 : contours_number = 1
BFC_20_015HT_60_14 : contours_number = 1
BFC_20_015HT_60_15 : 

In [64]:
a = pd.read_excel("./cut_by_detail/BFC/excel/cut_BFC_0710_new.xlsx")

In [40]:
b = pd.read_excel("./cut_by_detail/BFC/excel/cut_BFC_0710.xlsx")

In [65]:
a.describe()

,Unnamed: 0,threshold,axle_short,axle_long,ratio,angle,area_cv2,perimeter,contours_pixel,contours_number,calcularity
count,75068.000000,75068.0,75068.000000,75068.000000,75068.000000,75068.000000,75068.000000,75068.000000,75068.000000,75068.00000,75068.000000
mean,37533.500000,145.0,23.136225,28.411892,1.242640,89.177547,502.042555,90.839662,133.716709,41.65033,0.771130
std,21670.409341,0.0,3.336926,3.816824,0.189089,51.950778,126.772741,14.381733,8.773964,9.48218,0.126839
min,0.000000,145.0,15.000628,15.240824,1.000569,0.000000,4.500000,14.727922,0.000000,7.00000,0.074510
25%,18766.750000,145.0,20.926549,25.917195,1.109241,44.050604,422.000000,81.254833,127.737067,35.00000,0.731304
50%,37533.500000,145.0,23.074130,27.949211,1.193099,88.510265,490.000000,88.083260,132.747113,40.00000,0.816615
75%,56300.250000,145.0,25.226048,30.507307,1.322602,134.301373,569.500000,97.597979,138.482246,47.00000,0.855626
max,75067.000000,145.0,38.157505,39.999752,2.590457,179.994095,1171.000000,176.610171,221.793103,75.00000,0.929515


In [43]:
a.describe()

,Unnamed: 0,threshold,axle_short,axle_long,angle,area_cv2,perimeter,contours_pixel,contours_number,calcularity
count,61500.000000,61500.0,61500.000000,61500.000000,61500.000000,61500.000000,61500.000000,61500.000000,61500.000000,61500.000000
mean,30749.500000,145.0,23.075803,28.335279,89.034468,500.341561,91.909143,133.813909,43.185528,0.758175
std,17753.665115,0.0,3.386210,4.043852,51.830649,135.137906,17.785642,8.862291,11.292725,0.133715
min,0.000000,145.0,15.000628,16.035633,0.000000,4.500000,14.727922,85.087647,8.000000,0.064054
25%,15374.750000,145.0,20.844827,25.621951,44.032969,413.000000,79.941125,127.657807,35.000000,0.709940
50%,30749.500000,145.0,22.743059,27.737830,88.256283,475.500000,87.254833,132.826089,42.000000,0.806527
75%,46124.250000,145.0,25.026490,30.433408,133.983727,573.000000,100.325901,138.865758,49.000000,0.849853
max,61499.000000,145.0,38.520546,39.999752,179.991531,1171.000000,345.847759,221.793103,188.000000,0.929515


In [56]:
a[a['img_name'].str.contains("CYST")]

,Unnamed: 0,img_name,threshold,center,axle_short,axle_long,angle,area_cv2,perimeter,contours_pixel,contours_number,type,calcularity
1440,1440,BFC_20_052CYST_C_5_28,145,"(90.4561767578125, 59.92405700683594)",23.092707,28.815964,135.494659,503.5,88.669047,126.094203,44,HT,0.804350
1441,1441,BFC_20_052CYST_C_38_0,145,"(110.87757873535156, 255.90878295898438)",23.493618,31.472082,66.156418,543.5,94.811182,128.340813,53,HT,0.759399
1442,1442,BFC_20_052CYST_C_38_8,145,"(131.9725341796875, 166.0990447998047)",24.690893,26.693409,20.054245,512.0,87.254833,122.578526,42,HT,0.844657
1443,1443,BFC_20_052CYST_C_38_22,145,"(188.00274658203125, 84.17108917236328)",27.287312,28.805281,10.905669,606.5,95.154328,126.125220,47,HT,0.841324
1444,1444,BFC_20_052CYST_C_38_23,145,"(153.60594177246094, 82.0174560546875)",26.751873,29.627810,33.985935,615.0,94.911687,125.264368,46,HT,0.857483
...,...,...,...,...,...,...,...,...,...,...,...,...,...
57307,57307,BFC_20_048CYST_C_159_51,145,"(328.8063049316406, 132.07489013671875)",22.689329,28.844322,8.755643,506.5,85.840620,129.091991,38,Normal,0.863343
57308,57308,BFC_20_048CYST_C_159_59,145,"(62.241817474365234, 122.31519317626953)",24.793901,29.977707,25.970957,570.5,94.669047,124.501437,51,HT,0.799520
57309,57309,BFC_20_048CYST_C_159_67,145,"(339.8871765136719, 101.92141723632812)",22.705343,24.968424,59.720757,440.5,79.012193,126.412879,32,Normal,0.886232
57310,57310,BFC_20_048CYST_C_159_68,145,"(120.01551055908203, 104.25548553466797)",25.739983,32.787388,56.344009,655.5,98.325901,124.295833,42,HT,0.851582


In [72]:
z = [1,3,5,78,4,89]
sum(z) / len(z)

30.0

In [81]:
len(a['mass_name'].unique())

6264

In [78]:
df_group_mean_mean = pd.DataFrame()
for i in a['mass_name'].unique():
    mean = []
    for index, row in a.iterrows():
        if row['mass_name'] == i:
            mean.append(a.loc[index]["axle_short"])
    group_mean = sum(mean) / len(mean)
    
    data ={"mass_name" : i,"group_mean" : group_mean}
    
    df_group_mean = pd.concat([df_group_mean,pd.DataFrame([data])],ignore_index=True)


KeyboardInterrupt



In [106]:
a.describe()

,Unnamed: 0,threshold,axle_short,axle_long,ratio,angle,area_cv2,perimeter,contours_pixel,contours_number,calcularity
count,75068.000000,75068.0,75068.000000,75068.000000,75068.000000,75068.000000,75068.000000,75068.000000,75068.000000,75068.00000,75068.000000
mean,37533.500000,145.0,23.136225,28.411892,1.242640,89.177547,502.042555,90.839662,133.716709,41.65033,0.771130
std,21670.409341,0.0,3.336926,3.816824,0.189089,51.950778,126.772741,14.381733,8.773964,9.48218,0.126839
min,0.000000,145.0,15.000628,15.240824,1.000569,0.000000,4.500000,14.727922,0.000000,7.00000,0.074510
25%,18766.750000,145.0,20.926549,25.917195,1.109241,44.050604,422.000000,81.254833,127.737067,35.00000,0.731304
50%,37533.500000,145.0,23.074130,27.949211,1.193099,88.510265,490.000000,88.083260,132.747113,40.00000,0.816615
75%,56300.250000,145.0,25.226048,30.507307,1.322602,134.301373,569.500000,97.597979,138.482246,47.00000,0.855626
max,75067.000000,145.0,38.157505,39.999752,2.590457,179.994095,1171.000000,176.610171,221.793103,75.00000,0.929515


In [133]:
a_new = pd.DataFrame(a.groupby("mass_name")["axle_short"].mean()).reset_index()
a_new = pd.merge(a_new,pd.DataFrame(a.groupby("mass_name")["axle_long"].mean()).reset_index(),on=["mass_name"])
a_new = pd.merge(a_new,pd.DataFrame(a.groupby("mass_name")["ratio"].mean()).reset_index(),on=["mass_name"])
a_new = pd.merge(a_new,pd.DataFrame(a.groupby("mass_name")["angle"].mean()).reset_index(),on=["mass_name"])
a_new = pd.merge(a_new,pd.DataFrame(a.groupby("mass_name")["area_cv2"].mean()).reset_index(),on=["mass_name"])
a_new = pd.merge(a_new,pd.DataFrame(a.groupby("mass_name")["perimeter"].mean()).reset_index(),on=["mass_name"])
a_new = pd.merge(a_new,pd.DataFrame(a.groupby("mass_name")["contours_pixel"].mean()).reset_index(),on=["mass_name"])
a_new = pd.merge(a_new,pd.DataFrame(a.groupby("mass_name")["contours_number"].mean()).reset_index(),on=["mass_name"])
a_new = pd.merge(a_new,pd.DataFrame(a.groupby("mass_name")["calcularity"].mean()).reset_index(),on=["mass_name"])

In [134]:
a_new.columns = a_new.columns.str.replace(f'{a_new.columns.str}', f'group_mean_{a_new.columns.str}')

In [139]:
a_new.rename({"axle_short":"gp_mean_axle_short","axle_long":"gp_mean_axle_long",
              "ratio":"gp_mean_ratio","angle":"gp_mean_angle","area_cv2":"gp_mean_area_cv2",
              "perimeter":"gp_mean_perimeter","contours_pixel":"gp_mean_contours_pixel",
              "contours_number":"gp_mean_contours_number","calcularity":"gp_mean_calcularity"},axis=1,inplace=True)

In [140]:
a_new

,mass_name,gp_mean_axle_short,gp_mean_axle_long,gp_mean_ratio,gp_mean_angle,gp_mean_area_cv2,gp_mean_perimeter,gp_mean_contours_pixel,gp_mean_contours_number,gp_mean_calcularity
0,BFC_20_001_0,24.134674,29.475180,1.225324,73.525081,549.000000,91.551889,134.001799,40.200000,0.821743
1,BFC_20_001_1,25.051941,29.086228,1.176845,132.533379,566.200000,92.869047,130.858738,43.200000,0.820292
2,BFC_20_001_10,21.818149,28.407598,1.323123,114.111917,470.166667,84.789970,133.119614,37.888889,0.820259
3,BFC_20_001_100,26.504908,38.303268,1.445139,40.435860,761.000000,128.710677,130.858300,57.000000,0.576959
4,BFC_20_001_101,22.564141,28.924066,1.305118,83.421552,493.166667,89.035882,133.470841,38.833333,0.781584
...,...,...,...,...,...,...,...,...,...,...
6259,BFC_20_070_C_95,19.844316,27.127857,1.368739,95.378750,406.529412,83.976003,142.732740,37.588235,0.733669
6260,BFC_20_070_C_96,21.748290,26.885926,1.240048,172.894295,447.250000,83.547726,141.569270,38.000000,0.801705
6261,BFC_20_070_C_97,21.785825,26.828739,1.238467,82.078433,445.083333,89.823869,147.559254,40.000000,0.692382
6262,BFC_20_070_C_98,20.710950,27.355088,1.320864,115.155651,391.000000,95.497474,148.048695,41.000000,0.535234


In [128]:
a = a.drop("Unnamed: 0",axis=1)

In [144]:
pd.merge(a,a_new,on=["mass_name"],how="left").to_excel("./cut_by_detail/BFC/excel/merge_df.xlsx",index=False)

In [145]:
df = pd.read_excel("./cut_by_detail/BFC/excel/merge_df.xlsx")

In [146]:
df

,mass_name,img_name,threshold,center,axle_short,axle_long,ratio,angle,area_cv2,perimeter,...,gp_mean_calcularity,variance_axle_short,variance_axle_long,variance_ratio,variance_angle,variance_area_cv2,variance_perimeter,variance_contours_pixel,variance_contours_number,variance_calcularity
0,BFC_20_015HT_44,BFC_20_015HT_44_2,145,"(138.00047302246094, 161.76881408691406)",29.356182,30.644281,1.043878,89.317673,691.0,105.740114,...,0.808421,2.265153,0.670765,-0.065103,-23.888382,73.833333,7.852116,-2.869978,6.000000,-0.032195
1,BFC_20_015HT_44,BFC_20_015HT_44_17,145,"(211.1317596435547, 111.83262634277344)",28.191040,32.252995,1.144087,163.865814,671.5,103.497474,...,0.808421,1.100011,2.279478,0.035105,50.659760,54.333333,5.609476,-0.405468,5.000000,-0.021056
2,BFC_20_015HT_44,BFC_20_015HT_44_20,145,"(115.185546875, 90.6397933959961)",23.725866,27.023273,1.138979,86.434677,489.0,84.426406,...,0.808421,-3.365163,-2.950243,0.029998,-26.771378,-128.166667,-13.461592,3.275446,-11.000000,0.053251
3,BFC_20_015HT_60,BFC_20_015HT_60_8,145,"(106.1220932006836, 282.2130126953125)",20.488977,35.409039,1.728199,101.243988,527.0,92.426406,...,0.696152,-3.301205,1.683613,0.288609,11.515493,-65.200000,-11.079393,-5.371745,-13.400000,0.078681
4,BFC_20_015HT_60,BFC_20_015HT_60_16,145,"(169.03306579589844, 264.4440002441406)",26.963507,36.479099,1.352906,106.507652,757.0,115.396968,...,0.696152,3.173325,2.753674,-0.086684,16.779158,164.800000,11.891169,-2.099470,9.600000,0.017846
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75063,BFC_20_040_5,BFC_20_040_5_17,145,"(206.7635040283203, 168.34210205078125)",24.748724,32.483101,1.312516,56.559650,608.0,100.911687,...,0.776973,-0.883767,2.312030,0.130220,-48.473647,13.916667,2.235702,-4.758102,8.583333,-0.027061
75064,BFC_20_040_5,BFC_20_040_5_21,145,"(170.3862762451172, 133.4921112060547)",28.906200,32.815834,1.135252,131.858200,720.5,115.982755,...,0.776973,3.273710,2.644763,-0.047044,26.824902,126.416667,17.306770,3.366216,10.583333,-0.104249
75065,BFC_20_040_5,BFC_20_040_5_22,145,"(104.84925079345703, 130.37840270996094)",20.842018,27.637850,1.326064,167.414291,445.0,79.597979,...,0.776973,-4.790473,-2.533221,0.143768,62.380994,-149.083333,-19.078006,-2.395207,-17.416667,0.105183
75066,BFC_20_040_5,BFC_20_040_5_26,145,"(228.17486572265625, 92.60370635986328)",24.794304,30.052252,1.212063,157.703964,562.5,101.154328,...,0.776973,-0.838187,-0.118819,0.029766,52.670667,-31.583333,2.478343,4.759577,-0.416667,-0.086506


In [181]:
def test(data,col):
    test_df = data.groupby("mass_name")[col].describe().reset_index()
    test_df.rename({"mean": f'{col}_mean',"std": f'{col}_std'},axis=1,inplace=True)
    test_df = test_df[["mass_name",f'{col}_mean',f'{col}_std']]
    
    return test_df

In [182]:
test(df,"variance_axle_short")

,mass_name,variance_axle_short_mean,variance_axle_short_std
0,BFC_20_001_0,3.907985e-15,1.749661
1,BFC_20_001_1,-1.421085e-15,3.868277
2,BFC_20_001_10,2.763222e-15,2.315919
3,BFC_20_001_100,0.000000e+00,NaN
4,BFC_20_001_101,-4.144833e-15,2.579170
...,...,...,...
6259,BFC_20_070_C_95,1.044916e-15,2.647679
6260,BFC_20_070_C_96,0.000000e+00,0.701711
6261,BFC_20_070_C_97,-5.921189e-16,2.350220
6262,BFC_20_070_C_98,5.329071e-15,0.028454


In [185]:
df_new = test(df,"variance_axle_short")
df_new = pd.merge(df_new,test(df,"variance_axle_long"),on=["mass_name"])
df_new = pd.merge(df_new,test(df,"variance_ratio"),on=["mass_name"])
df_new = pd.merge(df_new,test(df,"variance_angle"),on=["mass_name"])
df_new = pd.merge(df_new,test(df,"variance_area_cv2"),on=["mass_name"])
df_new = pd.merge(df_new,test(df,"variance_perimeter"),on=["mass_name"])
df_new = pd.merge(df_new,test(df,"variance_contours_pixel"),on=["mass_name"])
df_new = pd.merge(df_new,test(df,"variance_contours_number"),on=["mass_name"])
df_new = pd.merge(df_new,test(df,"variance_calcularity"),on=["mass_name"])

In [186]:
df_new

,mass_name,variance_axle_short_mean,variance_axle_short_std,variance_axle_long_mean,variance_axle_long_std,variance_ratio_mean,variance_ratio_std,variance_angle_mean,variance_angle_std,variance_area_cv2_mean,variance_area_cv2_std,variance_perimeter_mean,variance_perimeter_std,variance_contours_pixel_mean,variance_contours_pixel_std,variance_contours_number_mean,variance_contours_number_std,variance_calcularity_mean,variance_calcularity_std
0,BFC_20_001_0,3.907985e-15,1.749661,3.552714e-16,1.409280,4.884981e-16,0.081761,-1.065814e-14,29.667064,0.000000e+00,58.123910,-8.526513e-15,5.346797,3.410605e-14,2.202263,-2.842171e-15,6.795423,-3.330669e-17,0.035830
1,BFC_20_001_1,-1.421085e-15,3.868277,3.197442e-15,3.571104,1.110223e-16,0.180284,-2.273737e-14,34.394083,-4.547474e-14,134.356532,0.000000e+00,12.421921,1.421085e-14,3.378160,-2.842171e-15,7.192589,-9.992007e-17,0.068634
2,BFC_20_001_10,2.763222e-15,2.315919,1.578984e-15,1.770642,2.960595e-16,0.230753,1.894781e-14,53.505287,-1.894781e-14,47.894415,-1.578984e-15,3.415592,6.315935e-15,3.111966,-3.947460e-15,4.400126,3.700743e-17,0.058335
3,BFC_20_001_100,0.000000e+00,NaN,0.000000e+00,NaN,0.000000e+00,NaN,0.000000e+00,NaN,0.000000e+00,NaN,0.000000e+00,NaN,0.000000e+00,NaN,0.000000e+00,NaN,0.000000e+00,NaN
4,BFC_20_001_101,-4.144833e-15,2.579170,-3.552714e-15,6.280948,-5.921189e-16,0.393457,0.000000e+00,73.981203,-1.894781e-14,108.056312,-1.184238e-14,11.877451,-4.736952e-14,4.919084,-9.473903e-15,4.792355,7.401487e-17,0.077494
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6259,BFC_20_070_C_95,1.044916e-15,2.647679,5.015596e-15,6.147073,2.873518e-16,0.238165,1.003119e-14,42.202037,-3.009357e-14,136.152542,2.507798e-15,16.803816,-3.343731e-14,6.593574,9.195259e-15,9.219943,3.591898e-17,0.142977
6260,BFC_20_070_C_96,0.000000e+00,0.701711,-3.552714e-15,4.275105,-1.110223e-16,0.236582,0.000000e+00,0.749844,0.000000e+00,67.528698,0.000000e+00,4.071068,1.421085e-14,4.252747,0.000000e+00,0.000000,0.000000e+00,0.043328
6261,BFC_20_070_C_97,-5.921189e-16,2.350220,3.552714e-15,1.443524,3.700743e-17,0.080181,2.368476e-15,37.339491,1.894781e-14,79.201273,7.105427e-15,9.054066,5.684342e-14,5.639371,0.000000e+00,6.324555,1.850372e-17,0.081259
6262,BFC_20_070_C_98,5.329071e-15,0.028454,5.329071e-15,1.783688,0.000000e+00,0.087938,1.421085e-14,44.158552,0.000000e+00,76.367532,0.000000e+00,4.828427,4.263256e-14,5.795588,0.000000e+00,1.414214,0.000000e+00,0.050987


In [189]:
df_all = pd.merge(df,df_new,on=["mass_name"],how="left")

In [194]:
df_all = df_all.fillna(0.0)

In [195]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75068 entries, 0 to 75067
Data columns (total 49 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   mass_name                      75068 non-null  object 
 1   img_name                       75068 non-null  object 
 2   threshold                      75068 non-null  int64  
 3   center                         75068 non-null  object 
 4   axle_short                     75068 non-null  float64
 5   axle_long                      75068 non-null  float64
 6   ratio                          75068 non-null  float64
 7   angle                          75068 non-null  float64
 8   area_cv2                       75068 non-null  float64
 9   perimeter                      75068 non-null  float64
 10  contours_pixel                 75068 non-null  float64
 11  contours_number                75068 non-null  int64  
 12  calcularity                    75068 non-null 

In [196]:
df_all.describe()

,threshold,axle_short,axle_long,ratio,angle,area_cv2,perimeter,contours_pixel,contours_number,calcularity,...,variance_area_cv2_mean,variance_area_cv2_std,variance_perimeter_mean,variance_perimeter_std,variance_contours_pixel_mean,variance_contours_pixel_std,variance_contours_number_mean,variance_contours_number_std,variance_calcularity_mean,variance_calcularity_std
count,75068.0,75068.000000,75068.000000,75068.000000,75068.000000,75068.000000,75068.000000,75068.000000,75068.00000,75068.000000,...,7.506800e+04,75068.000000,7.506800e+04,75068.000000,7.506800e+04,75068.000000,7.506800e+04,75068.000000,7.506800e+04,75068.000000
mean,145.0,23.136225,28.411892,1.242640,89.177547,502.042555,90.839662,133.716709,41.65033,0.771130,...,1.018469e-15,86.834584,-8.708096e-18,10.877460,-6.652228e-16,6.108148,-1.120221e-16,7.940148,-1.499662e-18,0.102028
std,0.0,3.336926,3.816824,0.189089,51.950778,126.772741,14.381733,8.773964,9.48218,0.126839,...,3.262372e-14,42.729938,1.154921e-14,4.786814,3.196023e-14,3.364685,4.566661e-15,2.549531,4.701043e-17,0.051574
min,145.0,15.000628,15.240824,1.000569,0.000000,4.500000,14.727922,0.000000,7.00000,0.074510,...,-8.687390e-14,0.000000,-7.105427e-14,0.000000,-7.815970e-14,0.000000,-1.053563e-14,0.000000,-1.332268e-16,0.000000
25%,145.0,20.926549,25.917195,1.109241,44.050604,422.000000,81.254833,127.737067,35.00000,0.731304,...,-1.894781e-14,57.906255,-3.617308e-15,7.417486,-2.583792e-14,4.095797,-1.937844e-15,6.278722,-3.647876e-17,0.064233
50%,145.0,23.074130,27.949211,1.193099,88.510265,490.000000,88.083260,132.747113,40.00000,0.816615,...,0.000000e+00,80.078809,0.000000e+00,10.406175,0.000000e+00,5.214362,0.000000e+00,7.669636,0.000000e+00,0.096045
75%,145.0,25.226048,30.507307,1.322602,134.301373,569.500000,97.597979,138.482246,47.00000,0.855626,...,2.094231e-14,106.142857,4.179663e-15,13.735710,2.447425e-14,6.899278,1.917338e-15,9.431501,3.172066e-17,0.131278
max,145.0,38.157505,39.999752,2.590457,179.994095,1171.000000,176.610171,221.793103,75.00000,0.929515,...,8.666292e-14,597.858783,5.968559e-14,58.870057,8.052818e-14,108.450769,1.053563e-14,37.476659,1.332268e-16,0.497304


In [197]:
df_all.to_excel("./cut_by_detail/BFC/excel/df_all.xlsx")